Reference
https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70

In [60]:
# import all the libraries
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import numpy as np
import pandas as pd

nltk.download('punkt')
from nltk.tokenize import sent_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Prachi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Prachi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df_articles = pd.read_csv('df_total.csv',encoding='cp1252')
df_articles.head()

,KEYWORDS,SUMMARY,TEXT,TITLE,URL,Unnamed: 0
0,"['young', 'write', 'making', 'worldsubscribe',...","At 15, the young artist would sit in her bedro...",Audio clip: Listen to audio clip.\n\nIn this e...,Peppina: Making Make-Believe Believable,https://radio.foxnews.com/2017/11/03/peppina-j...,0.0
1,"['korean', 'defectors', 'vegetation', 'sixth',...","- North Korean defector""I heard from a relativ...",North Korea’s crippling nuclear test site is t...,North Korea's nuclear test site causing 'defor...,http://www.foxnews.com/world/2017/11/07/north-...,1.0
2,"['johnson', 'political', 'lbj', 'war', 'texas'...",When Vice president Lyndon Baines Johnson acco...,Audio clip: Listen to audio clip.\n\nWhen Vice...,The Life And Times Of Lyndon Baines Johnson,https://radio.foxnews.com/2017/10/10/the-life-...,2.0
3,"['tell', 'exposed', 'scandal', 'lies', 'brazil...",NaN,NaN,Exposed: The lies journalists tell in the Braz...,https://video.foxnews.com/v/5638684128001/?pla...,3.0
4,"['ill', 'thanksgiving', '14000', 'thompson', '...",Thousands of strangers have responded to a ter...,Thousands of strangers have responded to a ter...,Terminally ill boy with wish for Christmas car...,https://www.foxnews.com/health/2017/11/07/term...,4.0


In [11]:
print(df_articles.isnull().sum())

KEYWORDS           0
SUMMARY       113700
TEXT          113600
TITLE            675
URL                0
Unnamed: 0         0
dtype: int64


In [14]:
df_articles.dropna(inplace=True)
df_articles.shape

(555510, 6)

In [201]:
#Rough Work - IGNORE
import string
sentences = sent_tokenize(df_articles['TEXT'][0])
sentences
stop_words = stopwords.words('english')
sen=[]
for sentence in sentences:
    #sen.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    #sentence = sentence.replace("[^a-zA-Z]", " ")
    #sentence = sentence.translate(str.maketrans(string.punctuation,' '*len(string.punctuation)))
    sentence = sentence.translate(str.maketrans('','',string.punctuation))
    #for word in sentence:
     #   word = word.strip()
    #    print("w: ",word)
    #
    #print("sent: ",sentence)
    #for word in sentence.split(" "):
    #    word = word.strip()
    #    print(word)
        #sen.append(word)
    #print("len ",(sentence.split()))
    sen.append([word for word in sentence.split(" ") if word not in stop_words])
    
#df = df_articles.iloc[0:10,:]
len(sen[0])

5

In [176]:
stop_words = stopwords.words('english')
sen_new = " ".join([i for i in sentences if i not in stop_words])
sen_new

'Audio clip: Listen to audio clip. In this episode, we meet Peppina, a Finnish singer/songwriter turned pop star who was afraid to share her music with her own community, so instead she went online and played her songs for the rest of the world. Subscribe and download on your favorite podcast platform: Apple Podcasts, Google Play, Tune In, Stitcher. At 15, the young artist would sit in her bedroom where she would write and post songs on HitRecord, an online production company founded by the actor and director Joseph Gordon-Levitt. There she would attract the attention of artists around the globe, quickly becoming the most downloaded musician working on the site. The producer, Brad Newman hosts this episode that follows a girl from a small island in Helsinki to the biggest stages in America. As one of her songs suggests, Peppina "makes make-believe believable." Like us on Facebook. Follow us on Twitter and Instagram. We want to know what you think. Pitch us an artist, or simply spread t

In [142]:
import re
stop_words = set(stopwords.words("english"))
def read_article(article):
    article_sentences = sent_tokenize(article)
    sentences = []
    
    for sentence in article_sentences:
        #sentence = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', sentence)
        sentence = sentence.lower()
        #print("sent ",sentence)
        #sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
        #sentence = sentence.replace("[^a-zA-Z]", " ")
        sentence = entence.translate(str.maketrans('','',string.punctuation))
        sentences.append([word for word in sentence.split(" ") if word not in stop_words])
        #sentences.append(sentence.split(" "))
    return sentences

In [116]:
def sentence_similarity(sent1,sent2):
    stops = set(stopwords.words("english"))
    all_words = list(set(sent1+sent2))
    
    vector1 = np.zeros([len(all_words)])
    vector2 = np.zeros([len(all_words)])
    
    #build vector for the first sentence
    for word in sent1:
        if word in stops:
            continue
        vector1[all_words.index(word)] += 1
        
    # build the vector for the second sentence
    for word in sent2:
        if word in stops:
            continue
        vector2[all_words.index(word)] += 1
 
    #print("cosine distance between {0} and {1} is {2}".format(vector1,vector2,1 - cosine_distance(vector1, vector2)))
    #NLTK implements cosine_distance, which is 1 - cosine_similarity
    return 1 - cosine_distance(vector1, vector2)

In [136]:
def build_similarity_matrix(sentences):
    similarity_matrix = np.zeros([len(sentences),len(sentences)])
    
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2])
            
    return np.asmatrix(similarity_matrix)

In [148]:
import operator
import collections

def generate_summary(top_n=5):
    
    for index,row in df_articles.iterrows():
        summarize_sentences = []
        summary=[]
        
        text = str(df_articles.loc[index,'TEXT'])
        #text = str(df_articles['TEXT'][0])
        sentences = read_article(text)
    
        # Step 2 - Generate Similary Martix across sentences
        sentence_similarity_matrix = build_similarity_matrix(sentences)
        #print("sent sim mat ",sentence_similarity_matrix)

        # Step 3 - Rank sentences in similarity martix
        sentence_similarity_graph = nx.from_numpy_matrix(sentence_similarity_matrix)
        #print("sent sim graph",sentence_similarity_graph)
        scores = nx.pagerank(sentence_similarity_graph)
        #print("scores",scores)

         # Step 4 - Sort the rank and pick top sentences
        #ranked_sentences = sorted(((scores[i,s]) for i,s in enumerate(sentences)), reverse=True)
        ranked_sentences = sorted(scores.items(), key=operator.itemgetter(1),reverse=True)[:top_n]
        ranked_dict = collections.OrderedDict(ranked_sentences)
        #print("Indexes of top ranked_sentence order are ", ranked_dict)   

        for key in ranked_dict.keys():
            summarize_sentences.append(" ".join(sentences[key]))
        summary.append(" ".join(summarize_sentences))
        
        #df_articles.loc[index,'SUMMARY_GEN'] = summary[0]
        return df_articles

In [151]:
import datetime
print("start of generating summary ",datetime.datetime.now())
df = generate_summary()
print("end of generating summary ",datetime.datetime.now())
df.to_csv("df_summary.csv",index=False)

start of generating summary  2019-04-18 11:06:51.747567
sent  audio clip: listen to audio clip.
sent  in this episode, we meet peppina, a finnish singer/songwriter turned pop star who was afraid to share her music with her own community, so instead she went online and played her songs for the rest of the world.
sent  subscribe and download on your favorite podcast platform: apple podcasts, google play, tune in, stitcher.
sent  at 15, the young artist would sit in her bedroom where she would write and post songs on hitrecord, an online production company founded by the actor and director joseph gordon-levitt.
sent  there she would attract the attention of artists around the globe, quickly becoming the most downloaded musician working on the site.
sent  the producer, brad newman hosts this episode that follows a girl from a small island in helsinki to the biggest stages in america.
sent  as one of her songs suggests, peppina "makes make-believe believable."
sent  like us on facebook.
sen

In [ ]:
Approach 2

In [210]:
import re
stop_words = set(stopwords.words("english"))
def read_article(article):
    article_sentences = sent_tokenize(article)
    sentences = [] 
    for sentence in article_sentences:
        sentence = sentence.lower()
        sentence = sentence.translate(str.maketrans('','',string.punctuation))
        sentences.append([word for word in sentence.split(" ") if word not in stop_words])
    return sentences

In [191]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [220]:
def create_sentence_vectors(clean_sentence):
    sentence_vectors=[]
    for sentence in clean_sentence:
        #if len(word) != 0:
        print("sent ",sentence)
        vec = sum([word_embeddings.get(word, np.zeros((100,))) for word in sentence])/(len(sentence)+0.001)
        #else:
           # vec = np.zeros((100,))
        sentence_vectors.append(vec)
    return sentence_vectors

In [203]:
from sklearn.metrics.pairwise import cosine_similarity
def create_similarity_matrix(clean_sentence,sentence_vectors):
    similarity_matrix = np.zeros([len(clean_sentence),len(clean_sentence)])
    for idx1 in range(len(clean_sentence)):
        for idx2 in range(len(clean_sentence)):
            if idx1 != idx2:
                similarity_matrix[idx1][idx2] = cosine_similarity(sentence_vectors[idx1].reshape(1,100), sentence_vectors[idx2].reshape(1,100))[0,0]      
    return np.asmatrix(similarity_matrix) 

In [205]:
import operator
import collections

def generate_summary(top_n=5):
    
    for index,row in df.iterrows():
        summarize_sentences = []
        summary=[]
        
        text = str(df.loc[index,'TEXT'])
        #text = str(df_articles['TEXT'][0])
        clean_sentences = read_article(text)
        
        sentence_vectors = create_sentence_vectors(clean_sentences)
        sentence_similarity_matrix = create_similarity_matrix(clean_sentences,sentence_vectors)
        
        # Step 2 - Generate Similary Martix across sentences
        #sentence_similarity_matrix = build_similarity_matrix(sentences)
        #print("sent sim mat ",sentence_similarity_matrix)

        # Step 3 - Rank sentences in similarity martix
        sentence_similarity_graph = nx.from_numpy_matrix(sentence_similarity_matrix)
        #print("sent sim graph",sentence_similarity_graph)
        scores = nx.pagerank(sentence_similarity_graph)
        #print("scores",scores)

         # Step 4 - Sort the rank and pick top sentences
        #ranked_sentences = sorted(((scores[i,s]) for i,s in enumerate(sentences)), reverse=True)
        ranked_sentences = sorted(scores.items(), key=operator.itemgetter(1),reverse=True)[:top_n]
        ranked_dict = collections.OrderedDict(ranked_sentences)
        #print("Indexes of top ranked_sentence order are ", ranked_dict)   

        for key in ranked_dict.keys():
            summarize_sentences.append(" ".join(sentences[key]))
        summary.append(" ".join(summarize_sentences))
        
        #df_articles.loc[index,'SUMMARY_GEN'] = summary[0]
        return df

In [221]:
print("start of generating summary ",datetime.datetime.now())
df = generate_summary()
print("end of generating summary ",datetime.datetime.now())
df

start of generating summary  2019-04-18 23:38:51.437347
sent  ['audio', 'clip', 'listen', 'audio', 'clip']
sent  ['episode', 'meet', 'peppina', 'finnish', 'singersongwriter', 'turned', 'pop', 'star', 'afraid', 'share', 'music', 'community', 'instead', 'went', 'online', 'played', 'songs', 'rest', 'world']
sent  ['subscribe', 'download', 'favorite', 'podcast', 'platform', 'apple', 'podcasts', 'google', 'play', 'tune', 'stitcher']
sent  ['15', 'young', 'artist', 'would', 'sit', 'bedroom', 'would', 'write', 'post', 'songs', 'hitrecord', 'online', 'production', 'company', 'founded', 'actor', 'director', 'joseph', 'gordonlevitt']
sent  ['would', 'attract', 'attention', 'artists', 'around', 'globe', 'quickly', 'becoming', 'downloaded', 'musician', 'working', 'site']
sent  ['producer', 'brad', 'newman', 'hosts', 'episode', 'follows', 'girl', 'small', 'island', 'helsinki', 'biggest', 'stages', 'america']
sent  ['one', 'songs', 'suggests', 'peppina', 'makes', 'makebelieve', 'believable']
sent  [

,KEYWORDS,SUMMARY,TEXT,TITLE,URL,Unnamed: 0,SUMMARY_GEN
0,"['young', 'write', 'making', 'worldsubscribe',...","At 15, the young artist would sit in her bedro...",Audio clip: Listen to audio clip.\n\nIn this e...,Peppina: Making Make-Believe Believable,https://radio.foxnews.com/2017/11/03/peppina-j...,0.0,"At 15, the young artist would sit in her bedro..."
1,"['korean', 'defectors', 'vegetation', 'sixth',...","- North Korean defector""I heard from a relativ...",North Korea’s crippling nuclear test site is t...,North Korea's nuclear test site causing 'defor...,http://www.foxnews.com/world/2017/11/07/north-...,1.0,The Research Association of Vision of North Ko...
2,"['johnson', 'political', 'lbj', 'war', 'texas'...",When Vice president Lyndon Baines Johnson acco...,Audio clip: Listen to audio clip.\n\nWhen Vice...,The Life And Times Of Lyndon Baines Johnson,https://radio.foxnews.com/2017/10/10/the-life-...,2.0,Then an assassin's bullet forever changed Lynd...
4,"['ill', 'thanksgiving', '14000', 'thompson', '...",Thousands of strangers have responded to a ter...,Thousands of strangers have responded to a ter...,Terminally ill boy with wish for Christmas car...,https://www.foxnews.com/health/2017/11/07/term...,4.0,“We also have continued to open Christmas card...
5,"['signs', 'political', 'analysis', 'fnc', 'net...","NEW YORK – SEPTEMBER 25, 2017 – FOX News Chann...","NEW YORK – SEPTEMBER 25, 2017 – FOX News Chann...",FOX NEWS CHANNEL SIGNS GIANNO CALDWELL AS POLI...,http://press.foxnews.com/2017/09/fox-news-chan...,5.0,FOX News Channel (FNC) is a 24-hour all-encomp...


In [223]:
df.to_csv("df_summary_extractive_approach2.csv",index=False)

In [204]:
df = df_articles.head()
df

,KEYWORDS,SUMMARY,TEXT,TITLE,URL,Unnamed: 0,SUMMARY_GEN
0,"['young', 'write', 'making', 'worldsubscribe',...","At 15, the young artist would sit in her bedro...",Audio clip: Listen to audio clip.\n\nIn this e...,Peppina: Making Make-Believe Believable,https://radio.foxnews.com/2017/11/03/peppina-j...,0.0,"At 15, the young artist would sit in her bedro..."
1,"['korean', 'defectors', 'vegetation', 'sixth',...","- North Korean defector""I heard from a relativ...",North Korea’s crippling nuclear test site is t...,North Korea's nuclear test site causing 'defor...,http://www.foxnews.com/world/2017/11/07/north-...,1.0,The Research Association of Vision of North Ko...
2,"['johnson', 'political', 'lbj', 'war', 'texas'...",When Vice president Lyndon Baines Johnson acco...,Audio clip: Listen to audio clip.\n\nWhen Vice...,The Life And Times Of Lyndon Baines Johnson,https://radio.foxnews.com/2017/10/10/the-life-...,2.0,Then an assassin's bullet forever changed Lynd...
4,"['ill', 'thanksgiving', '14000', 'thompson', '...",Thousands of strangers have responded to a ter...,Thousands of strangers have responded to a ter...,Terminally ill boy with wish for Christmas car...,https://www.foxnews.com/health/2017/11/07/term...,4.0,“We also have continued to open Christmas card...
5,"['signs', 'political', 'analysis', 'fnc', 'net...","NEW YORK – SEPTEMBER 25, 2017 – FOX News Chann...","NEW YORK – SEPTEMBER 25, 2017 – FOX News Chann...",FOX NEWS CHANNEL SIGNS GIANNO CALDWELL AS POLI...,http://press.foxnews.com/2017/09/fox-news-chan...,5.0,FOX News Channel (FNC) is a 24-hour all-encomp...


### ROUGH WORK BELOW - IGNORE

In [154]:
#df = df_articles.iloc[0:10,:]
#df.head()

df['SUMMARY_GEN'][1]

'The Research Association of Vision of North Korea interviewed 21 North Korean defectors who lived in Kilju, a nearby town north of the Punngye-ri nuclear test site where six tests have been conducted, and said babies were reportedly being born with birth defects and residents feared radiation contamination because of the high mortality rate for any form of life, South Korean newspaper Chosun Ilbo reported. North Korea’s crippling nuclear test site is turning the area into a “wasteland” where “deformed babies” are being born and 80 percent of vegetation dies off due to nuclear radiation, nearly two dozen defectors told a South Korean newspaper Monday. The defector accounts by Chosun Ilbo were not verified by North Korean officials, but the former North Korean residents claimed Kim Jong Un’s regime made sure none of the local residents were able to tell their stories. A defector who fled North Korea in 2010 recalled how only soldiers and their family members were evacuated before two te

In [155]:
df['SUMMARY'][1]

'- North Korean defector"I heard from a relative in Kilju that deformed babies were born in hospitals there,” one defector said, according to the newspaper.\nOne person recalled, "Kilju locals who made appointments in a large hospital in Pyongyang were not allowed to enter the capital after the sixth nuclear test."\nScientists believe the site was destabilized after the sixth nuclear test on Sept. 3, when a purported hydrogen bomb was detonated.\nSatellite images obtained by 38 North, which specialized in North Korean issues, also showed a possible “collapsed chimney crater” at Mount Mantap.\nDr. Joel Myers, the founder and chairman of AccuWeather, said lingering radiation from the sixth nuclear test could threaten other countries.'

In [156]:
df['TEXT'][1]

'North Korea’s crippling nuclear test site is turning the area into a “wasteland” where “deformed babies” are being born and 80 percent of vegetation dies off due to nuclear radiation, nearly two dozen defectors told a South Korean newspaper Monday.\n\nThe Research Association of Vision of North Korea interviewed 21 North Korean defectors who lived in Kilju, a nearby town north of the Punngye-ri nuclear test site where six tests have been conducted, and said babies were reportedly being born with birth defects and residents feared radiation contamination because of the high mortality rate for any form of life, South Korean newspaper Chosun Ilbo reported.\n\n“I personally saw corpses floating down the river with their limbs severed." - North Korean defector\n\n"I heard from a relative in Kilju that deformed babies were born in hospitals there,” one defector said, according to the newspaper.\n\nAnother added: "I spoke on the phone with family members I left behind there and they told me 

In [126]:
df[['SUMMARY','SUMMARY_GEN']]

,SUMMARY,SUMMARY_GEN
0,"At 15, the young artist would sit in her bedro...","At 15, the young artist would sit in her bedro..."
1,"- North Korean defector""I heard from a relativ...",The Research Association of Vision of North Ko...
2,When Vice president Lyndon Baines Johnson acco...,Then an assassin's bullet forever changed Lynd...
4,Thousands of strangers have responded to a ter...,“We also have continued to open Christmas card...
5,"NEW YORK – SEPTEMBER 25, 2017 – FOX News Chann...",FOX News Channel (FNC) is a 24-hour all-encomp...
6,Democrat Ralph Northam won Virginia’s gubernat...,The Northam campaign called Gillespie’s ads “b...
7,If federal employees cannot perform the simple...,If our government cannot perform simple tasks ...
8,No stranger to making an entrance with her chi...,No stranger to making an entrance with her chi...
9,At least 26 people were killed and 20 wounded ...,At least 26 people were killed and 20 wounded ...
10,We’re now living in a violent society where ev...,"In fact, most of the 10,000 or so people who a..."


In [127]:
df.to_csv("df_gen_summary.csv",index=False)

In [105]:
generate_summary()
#summary

sent  Audio clip  Listen to audio clip 
sent  In this episode  we meet Peppina  a Finnish singer songwriter turned pop star who was afraid to share her music with her own community  so instead she went online and played her songs for the rest of the world 
sent  Subscribe and download on your favorite podcast platform  Apple Podcasts  Google Play  Tune In  Stitcher 
sent  At 15  the young artist would sit in her bedroom where she would write and post songs on HitRecord  an online production company founded by the actor and director Joseph Gordon Levitt 
sent  There she would attract the attention of artists around the globe  quickly becoming the most downloaded musician working on the site 
sent  The producer  Brad Newman hosts this episode that follows a girl from a small island in Helsinki to the biggest stages in America 
sent  As one of her songs suggests  Peppina  makes make believe believable  
sent  Like us on Facebook 
sent  Follow us on Twitter and Instagram 
sent  We want to 

'As one of her songs suggests  Peppina  makes make believe believable  .Subscribe and download on your favorite podcast platform  Apple Podcasts  Google Play  Tune In  Stitcher . childrenofsong.In this episode  we meet Peppina  a Finnish singer songwriter turned pop star who was afraid to share her music with her own community  so instead she went online and played her songs for the rest of the world .Pitch us an artist  or simply spread the word and find out more about the series '

In [110]:
 generate_summary()

sent  Audio clip: Listen to audio clip.
sent  In this episode, we meet Peppina, a Finnish singer/songwriter turned pop star who was afraid to share her music with her own community, so instead she went online and played her songs for the rest of the world.
sent  Subscribe and download on your favorite podcast platform: Apple Podcasts, Google Play, Tune In, Stitcher.
sent  At 15, the young artist would sit in her bedroom where she would write and post songs on HitRecord, an online production company founded by the actor and director Joseph Gordon-Levitt.
sent  There she would attract the attention of artists around the globe, quickly becoming the most downloaded musician working on the site.
sent  The producer, Brad Newman hosts this episode that follows a girl from a small island in Helsinki to the biggest stages in America.
sent  As one of her songs suggests, Peppina "makes make-believe believable."
sent  Like us on Facebook.
sent  Follow us on Twitter and Instagram.
sent  We want to 

'At 15, the young artist would sit in her bedroom where she would write and post songs on HitRecord, an online production company founded by the actor and director Joseph Gordon-Levitt. Like us on Facebook. Follow us on Twitter and Instagram. In this episode, we meet Peppina, a Finnish singer/songwriter turned pop star who was afraid to share her music with her own community, so instead she went online and played her songs for the rest of the world. Pitch us an artist, or simply spread the word and find out more about the series.'

In [111]:
df_articles['SUMMARY'][0]

'At 15, the young artist would sit in her bedroom where she would write and post songs on HitRecord, an online production company founded by the actor and director Joseph Gordon-Levitt.\nThere she would attract the attention of artists around the globe, quickly becoming the most downloaded musician working on the site.\nThe producer, Brad Newman hosts this episode that follows a girl from a small island in Helsinki to the biggest stages in America.\nAs one of her songs suggests, Peppina "makes make-believe believable."\nPitch us an artist, or simply spread the word and find out more about the series.'

In [113]:
sentences

['Audio clip: Listen to audio clip.',
 'In this episode, we meet Peppina, a Finnish singer/songwriter turned pop star who was afraid to share her music with her own community, so instead she went online and played her songs for the rest of the world.',
 'Subscribe and download on your favorite podcast platform: Apple Podcasts, Google Play, Tune In, Stitcher.',
 'At 15, the young artist would sit in her bedroom where she would write and post songs on HitRecord, an online production company founded by the actor and director Joseph Gordon-Levitt.',
 'There she would attract the attention of artists around the globe, quickly becoming the most downloaded musician working on the site.',
 'The producer, Brad Newman hosts this episode that follows a girl from a small island in Helsinki to the biggest stages in America.',
 'As one of her songs suggests, Peppina "makes make-believe believable."',
 'Like us on Facebook.',
 'Follow us on Twitter and Instagram.',
 'We want to know what you think.',

In [147]:
np.zeros([len(s)])
#len(s)

array([0., 0., 0., 0., 0., 0., 0.])

In [86]:

d = {0: 12, 2: 14, 3: 13, 4: 11, 5: 10}


sorted_d = sorted(d.items(), key=operator.itemgetter(1),reverse=True)[:3]
sorted_d

[(2, 14), (3, 13), (0, 12)]

In [90]:
#sentences[]

sorted_dict = collections.OrderedDict(sorted_d)
for key in sorted_dict.keys():
    #print(sentences[key])
    print(key)

2
3
0
